# Schemas

> Schema registry and management for settings

In [ ]:
#| default_exp core.schemas

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from pathlib import Path
from typing import Dict, Any, Optional, Union

Provides a centralized place to register and access settings schemas. Supports both individual schemas and `SchemaGroup` objects for organizing related configurations.

In [ ]:
#| export
class SettingsRegistry:
    """Registry for managing settings schemas and schema groups."""
    
    def __init__(self):
        self._schemas: Dict[str, Union[Dict[str, Any], 'SchemaGroup']] = {}
    
    def register(
        self,
        schema: Union[Dict[str, Any], 'SchemaGroup'],  # Schema or SchemaGroup to register
        name: Optional[str] = None  # Optional name override
    ):
        """Register a settings schema or schema group."""
        # Import here to avoid circular dependency
        from cjm_fasthtml_settings.core.schema_group import SchemaGroup
        
        if isinstance(schema, SchemaGroup):
            schema_name = schema.name
        else:
            schema_name = name or schema.get('name')
            if not schema_name:
                raise ValueError("Schema must have a 'name' field or name must be provided")
        
        self._schemas[schema_name] = schema
    
    def get(
        self,
        name: str  # Name of the schema/group to retrieve
    ) -> Optional[Union[Dict[str, Any], 'SchemaGroup']]:  # The schema/group, or None if not found
        """Get a registered schema or group by name."""
        return self._schemas.get(name)
    
    def list_schemas(
        self
    ) -> list:  # List of registered schema/group names
        """List all registered schema and group names."""
        return list(self._schemas.keys())
    
    def get_all(
        self
    ) -> Dict[str, Union[Dict[str, Any], 'SchemaGroup']]:  # All schemas and groups
        """Get all registered schemas and groups."""
        return self._schemas.copy()
    
    def resolve_schema(
        self,
        id: str  # Schema ID (can be 'name' or 'group_schema' format)
    ) -> tuple:  # (schema_dict, error_message)
        """Resolve a schema ID to a schema dictionary."""
        from cjm_fasthtml_settings.core.schema_group import SchemaGroup
        
        # Try direct lookup first
        item = self._schemas.get(id)
        if item:
            if isinstance(item, SchemaGroup):
                return None, f"'{id}' is a group, not a schema. Use 'group_schemaname' format."
            return item, None
        
        # Try grouped schema lookup (format: group_schema)
        if "_" in id:
            group_name, schema_key = id.split("_", 1)
            group = self._schemas.get(group_name)
            
            if isinstance(group, SchemaGroup):
                schema = group.get_schema(schema_key)
                if schema:
                    # Add the unique_id to the schema for proper saving
                    return {**schema, "unique_id": id}, None
                else:
                    return None, f"Schema '{schema_key}' not found in group '{group_name}'"
        
        return None, f"Settings '{id}' not found"

In [ ]:
# Example: Create and use a registry
from cjm_fasthtml_settings.core.config import get_app_config_schema

registry = SettingsRegistry()

# Register the app config schema
app_schema = get_app_config_schema(app_title="My App")
registry.register(app_schema)

# Register a custom schema
custom_schema = {
    "name": "custom",
    "title": "Custom Settings",
    "type": "object",
    "properties": {
        "api_key": {
            "type": "string",
            "title": "API Key",
            "default": ""
        }
    }
}
registry.register(custom_schema)

print(f"Registered schemas: {registry.list_schemas()}")
print(f"\nGeneral schema title: {registry.get('general')['title']}")
print(f"Custom schema title: {registry.get('custom')['title']}")

Registered schemas: ['general', 'custom']

General schema title: Application Configuration
Custom schema title: Custom Settings


In [ ]:
# Example: Register and use SchemaGroups
from cjm_fasthtml_settings.core.schema_group import SchemaGroup

registry2 = SettingsRegistry()

# Register a simple schema
registry2.register({
    "name": "general",
    "title": "General Settings",
    "type": "object",
    "properties": {"app_name": {"type": "string", "default": "My App"}}
})

# Register a schema group
media_group = SchemaGroup(
    name="media",
    title="Media Settings",
    schemas={
        "scanner": {
            "name": "scanner",
            "title": "Scanner Settings",
            "type": "object",
            "properties": {"scan_path": {"type": "string", "default": "/media"}}
        },
        "player": {
            "name": "player",
            "title": "Player Settings",
            "type": "object",
            "properties": {"volume": {"type": "integer", "default": 50}}
        }
    }
)
registry2.register(media_group)

print(f"Registered items: {registry2.list_schemas()}")
print(f"\nDirect schema lookup:")
schema, err = registry2.resolve_schema("general")
print(f"  'general' -> {schema['title'] if schema else err}")

print(f"\nGrouped schema lookup:")
schema, err = registry2.resolve_schema("media_scanner")
print(f"  'media_scanner' -> {schema['title'] if schema else err}")
print(f"  unique_id: {schema.get('unique_id') if schema else 'N/A'}")

schema, err = registry2.resolve_schema("media_player")
print(f"  'media_player' -> {schema['title'] if schema else err}")

Registered items: ['general', 'media']

Direct schema lookup:
  'general' -> General Settings

Grouped schema lookup:
  'media_scanner' -> Scanner Settings
  unique_id: media_scanner
  'media_player' -> Player Settings


In [ ]:
# Example: Using the module-level registry
from cjm_fasthtml_settings.core.schemas import registry as settings_registry
from cjm_fasthtml_settings.core.config import get_app_config_schema

# Clear for demo (normally you wouldn't do this)
settings_registry._schemas = {}

# Register schemas
settings_registry.register(get_app_config_schema(app_title="My App", include_theme=False))
settings_registry.register({
    "name": "custom",
    "title": "Custom Settings",
    "type": "object",
    "properties": {"api_key": {"type": "string", "title": "API Key", "default": ""}}
})

print(f"Registered schemas: {settings_registry.list_schemas()}")
print(f"General schema: {settings_registry.get('general')['title']}")

Registered schemas: ['general', 'custom']
General schema: Application Configuration


## Module-Level Registry Instance

In [ ]:
#| export
# Module-level registry instance
# This is the single source of truth for all settings schemas
# Routes and other modules will import and use this instance
registry = SettingsRegistry()

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()